In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile,Aer,assemble, execute
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit_aer import AerSimulator

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive inside a session. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# with Session(backend=service.backend("ibmq_qasm_simulator")):
#     result = Sampler().run(circuits).result()

In [2]:
alice_bits = [0,1,1,0,0,0,1,0,1,1]
alice_choices = ['NS', 'NS', 'S', 'NS', 'S','NS', 'S', 'NS','NS', 'S']
encoded_qubits = [] #an empty list to append with circuits of single qubit.

for q in range(len(alice_bits)):
    qc = QuantumCircuit(1, 1)
    if alice_bits[q] == 1:
        qc.x(0)
    if alice_choices[q]=='S':
        qc.h(0)

    encoded_qubits.append(qc)

print("Alice created the key of 10 bits: " + str(alice_bits))
print("Alice Superposition choices:" + str(alice_choices))
print("Now, Alice 'sends' her qubits to Bob >>>>>>>>>>>>>>>>>>>>>>>>>>>>")



Alice created the key of 10 bits: [0, 1, 1, 0, 0, 0, 1, 0, 1, 1]
Alice Superposition choices:['NS', 'NS', 'S', 'NS', 'S', 'NS', 'S', 'NS', 'NS', 'S']
Now, Alice 'sends' her qubits to Bob >>>>>>>>>>>>>>>>>>>>>>>>>>>>


In [3]:

print("***************** IF EVE INTERCEPTS! ******************")

for q in range(len(encoded_qubits)):
    encoded_qubits[q].measure(0,0)

eve_bits = []

backend = Aer.get_backend('qasm_simulator')

for q in range(len(encoded_qubits)):

  job = execute(encoded_qubits[q], backend = backend, shots = 1, memory = True)
  result = job.result()
  measured_bit = int(result.get_memory()[0])

  eve_bits.append(measured_bit)

print("Eve measured bits: " + str(eve_bits))
print("Superposition will collapse and Bob will recives intercepted qubits.")
# as Qiskit does not recognize multiple measurements on the same qubit. So,
# EVE MUST PREPARE NEW QUBITS TO BE SENT BASED ON HER MEASUREMENTS

encoded_qubits = [] 

for q in range(len(eve_bits)):
    qc = QuantumCircuit(1, 1)
    if eve_bits[q] == 1:
        qc.x(0)

    encoded_qubits.append(qc)

***************** IF EVE INTERCEPTS! ******************
Eve measured bits: [0, 1, 1, 0, 0, 0, 1, 0, 1, 1]
Superposition will collapse and Bob will recives intercepted qubits.


In [4]:
bob_choices = ['S', 'NS', 'NS', 'NS', 'S','NS', 'NS', 'NS','S', 'S']

for q in range(len(bob_choices)):

    if bob_choices[q] == 'S':
        encoded_qubits[q].h(0)

    encoded_qubits[q].measure(0,0)
    
    
bob_bits = []

backend = Aer.get_backend('qasm_simulator')

for q in range(len(encoded_qubits)):

  job = execute(encoded_qubits[q], backend = backend, shots = 1, memory = True)
  result = job.result()
  measured_bit = int(result.get_memory()[0])

  bob_bits.append(measured_bit)

print("Bob Superposition choices before measuring: " + str(bob_choices))
print("bob measured: " + str(bob_bits))



Bob Superposition choices before measuring: ['S', 'NS', 'NS', 'NS', 'S', 'NS', 'NS', 'NS', 'S', 'S']
bob measured: [0, 1, 1, 0, 0, 0, 1, 0, 0, 0]


In [5]:
alice_key = []

for q in range(len(encoded_qubits)):
    if alice_choices[q] == bob_choices[q]:
        alice_key.append(alice_bits[q])
print("effective_alice_key",alice_key)
        
        
bob_key = []

for q in range(len(encoded_qubits)):
    if alice_choices[q] == bob_choices[q]:
       bob_key.append(bob_bits[q])
print("effective_bob_key",bob_key)
       

effective_alice_key [1, 0, 0, 0, 0, 1]
effective_bob_key [1, 0, 0, 0, 0, 0]


In [6]:
if alice_key == bob_key:
    print("SECURE! as both key match"+'key=='+ str(alice_key) )
else:
    print("EVE DETECTED! as key mismatch Let's use another quantum channel.")

EVE DETECTED! as key mismatch Let's use another quantum channel.


In [7]:
print("WITHOUT EVE'S INTERCEPTION")
print("$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")

alice_bits = [0,1,1,0,0,0,1,0,1,1]
alice_choices = ['NS', 'NS', 'S', 'NS', 'S','NS', 'S', 'NS','NS', 'S']
encoded_qubits = [] #an empty list that we will append with circuits

for q in range(len(alice_bits)):
    qc = QuantumCircuit(1, 1)
    if alice_bits[q] == 1:
        qc.x(0)
    if alice_choices[q]=='S':
        qc.h(0)

    encoded_qubits.append(qc)

print("Alice created the key of 10 bits: " + str(alice_bits))
print("Alice Superposition choices: " + str(alice_choices))
print("Now, Alice 'sends' her qubits to Bob through Quantum channel")
print("||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||")


bob_choices = ['S', 'NS', 'NS', 'NS', 'S','NS', 'NS', 'NS','S', 'S']

for q in range(len(bob_choices)):

    if bob_choices[q] == 'S':
        encoded_qubits[q].h(0)

    encoded_qubits[q].measure(0,0)
    
#-------------------------------------------------------------------------------------    
bob_bits = []

backend = Aer.get_backend('qasm_simulator')

for q in range(len(encoded_qubits)):

  job = execute(encoded_qubits[q], backend = backend, shots = 1, memory = True)
  result = job.result()
  measured_bit = int(result.get_memory()[0])

  bob_bits.append(measured_bit)

print("Bob Superposition choices before measuring: " + str(bob_choices))
print("Bob measured: " + str(bob_bits))
#------------------------------------------------------------------------------------
alice_key = []

for q in range(len(encoded_qubits)):
    if alice_choices[q] == bob_choices[q]:
        alice_key.append(alice_bits[q])
print("effective_alice_key",alice_key)
        
        
bob_key = []

for q in range(len(encoded_qubits)):
    if alice_choices[q] == bob_choices[q]:
       bob_key.append(bob_bits[q])
print("effective_bob_key",bob_key)

print("||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||")
if alice_key == bob_key:
    print("SECURE! as both key match "+'KEY=='+ str(alice_key) )
else:
    print("EVE DETECTED! as key mismatch Let's use another quantum channel.")


WITHOUT EVE'S INTERCEPTION
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Alice created the key of 10 bits: [0, 1, 1, 0, 0, 0, 1, 0, 1, 1]
Alice Superposition choices: ['NS', 'NS', 'S', 'NS', 'S', 'NS', 'S', 'NS', 'NS', 'S']
Now, Alice 'sends' her qubits to Bob through Quantum channel
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Bob Superposition choices before measuring: ['S', 'NS', 'NS', 'NS', 'S', 'NS', 'NS', 'NS', 'S', 'S']
Bob measured: [0, 1, 1, 0, 0, 0, 1, 0, 0, 1]
effective_alice_key [1, 0, 0, 0, 0, 1]
effective_bob_key [1, 0, 0, 0, 0, 1]
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
SECURE! as both key match KEY==[1, 0, 0, 0, 0, 1]
